In [50]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-



'''
MAKE SURE TO RETURN TYPE 'Individual' (toolbox.individual())

It is safe to mutate droprates/pooling stuff, etc. But inserting/removing seems impossible?
(Type 'Individual' has no function 'remove'/'append')
'''




from representations import INSERTABLE, MUTABLE_PARAMS, default_init_nn_repr, REPR_MAKERS
import representations
import numpy as np
import random

FILTER_MUTATIONS = np.array([16, 32, 64])

containerIndividual = None
getRandomLayer = None



def mutate_append_remove(reprs, prob_remove=1):
    #@Deprecated
    '''
    Randomly appends or removes a layer. 
        prob_remove: probability that it removes a layer
    Returns: mutated representation
    '''
    if np.random.random() < prob_remove:
        if len(reprs) > 0:
            reprs.remove(np.random.choice(reprs))
    else:
        reprs = insert_layer(reprs)
    return reprs

# -------------------------------------------------------



def setIndividual(container):
    '''
    Sets the individual Object to the mutations global class.
    In order to generate a representation of type 'Individual', you need to call the variable.
    Example:
        containerIndividual(array) --> Individual([block1], [block2], [block3])
    '''
    global containerIndividual
    containerIndividual = container

def setInitialization(func):
    global getRandomLayer
    getRandomLayer = func


def mutate_layer(layer, verbose=False):
    '''
    Looks up the initializer function for a type,
    and replaces it with a new initialization.

    Appended [0] because you need to access the container inside the
    Individual({type: conv2dpool, params})
    class.    
    '''
    for elem in REPR_MAKERS:
        if layer['type'] == elem:
            layer = REPR_MAKERS[elem]()
            if verbose:
                print('MUTATED LAYER %s' % layer['type'])
    return layer


def removeBlock(repr):
    #@Deprecated
    random.shuffle(repr)
    return repr.pop()


def mutate_network(reprRaw, mutations=2, verbose=False, appendRemoveProb = 0.6):
    '''
    Mutates a whole representation network.
    Arguments:
        appendRemoveProb: Probability that a block is being appended/removed. 
            The inverse is the probability of mutating a block itself (so not changing the structure)
    '''
    repr = reprRaw.tolist()

    if mutations > len(repr):
        mutations = len(repr) # prevents setting higher count of mutations than length of representation


    if verbose:
        print("MUTATING %d BLOCKS OF NETWORK" % mutations)
    
    
    
    if (np.random.random() < appendRemoveProb): #if random evaluates to do an append/remove

        #Appending / removing blocks:
        if np.random.random() < 0.5 or len(repr) <= 2 :
            print("APPEND BLOCK ELEM: ", len(repr))
            repr.append(getRandomLayer())
            print("NEW BLOCK ELEM: ", len(repr))
        else:
            print("REMOVE BLOCK ELEM: ", len(repr))
            random.shuffle(repr)
            repr.pop()
            print("NEW BLOCK ELEM: ", len(repr))

    else:

        #Mutating blocks itself:
        for layerIndex in np.random.randint(0, len(repr), mutations):
            if verbose:
                repr[layerIndex] = mutate_layer(repr[layerIndex], verbose=True)
            else:
                repr[layerIndex] = mutate_layer(repr[layerIndex])

        


    return containerIndividual(repr),

In [51]:
import random
import numpy as np
import array

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import representations, fitness, mutations
import sys
from pprint import pprint


#INITIAL_BLOCKS = 5 # Represents how many random layer blocks each NNet should start with
INITIAL_BLOCKS = 5 # Random Exclusive represents how many random layer blocks each NNet should start with

POPULATION = 10
GENERATIONS = 10
PROB_MUTATIONS = 0.23 # Probability of mutating in a new generation
PROB_MATE = 0.4 # Probability of mating / crossover in a new generation
NUMBER_EPOCHS = 5 #Epochs when training the network

#---------------------

def getRandomLayer():
    #Possible networks to choose from:
    '''
    networks = [
        [representations.make_conv2d_repr(),
        representations.make_pool_repr()],

        [representations.make_dropout_repr(),
        representations.make_conv2d_repr()],

        [representations.make_batchnorm_repr()],

        [representations.make_noise_repr()]
    ]

    probabilities = [0.3, 0.3, 0.25, 0.15]


    out = []


    for x in range(0,iterations):
        choice = np.random.choice(networks, p=probabilities)
        for layer in choice:
            out.append(layer)

    return out
    '''

    # get this list from representations.REPR_MAKERS. If you add a new block type,
    # also add it to the representations.REPR_MAKERS list, so it can also mutate.
    networks = [
        representations.make_conv2d_pool_repr(),
        representations.make_conv2d_dropout_repr(),
        representations.make_noise_repr(),
        representations.make_dropout_repr(),
    ]
    probabilities = np.array([0.5, 0.5, 0.1, 0.1])
    probabilities = probabilities / probabilities.sum()
    return np.random.choice(networks,p=probabilities)


'''
Evaluation function (should return the fitness)
'''
def evaluateFunc(individual):
    return fitness.evaluate_nn(individual, NUMBER_EPOCHS), #<--- IMPORTANT: add the comma ','; as it needs to return a tuple

def initRepeatRandom(container, func, n):
    """
    Extended toolbox.initRepeat() function to work with random initialization instead of fixed numbers.
    Set this length to be minimal of 2, as we do two point crossover!!
    """
    return container(func() for _ in range(np.random.randint(2,n)))

# -------------- Init / Main stuff ----------------------

# Create attributes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray,  fitness=creator.FitnessMax)

mutations.setIndividual(creator.Individual)
mutations.setInitialization(getRandomLayer)


toolbox = base.Toolbox()

toolbox.register("individual", initRepeatRandom, creator.Individual, getRandomLayer, n=INITIAL_BLOCKS) #<-- Creates 3 elements. This random, however does not evaluate the random function each time yet
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("evaluate", evaluateFunc) #register the evaluation function
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutations.mutate_network, verbose=True, mutations=2)
toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.register("select", tools.selBest)
#deap.tools.selBest(individuals, k, fit_attr='fitness')





c:\users\ruben\appdata\local\programs\python\python36\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
c:\users\ruben\appdata\local\programs\python\python36\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [52]:
random.seed(1337)
pop = toolbox.population(n=POPULATION)



hof = tools.HallOfFame(10, similar=np.array_equal)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [76]:
indiv = pop[3]

In [77]:
indiv

Individual([{'type': 'dropout', 'params': {'rate': 0.23}},
            {'type': 'conv2ddropout', 'params': {'filters': 16, 'kernel_size': 3, 'activation': 'relu', 'rate': 0.29}},
            {'type': 'conv2dpool', 'params': {'filters': 8, 'kernel_size': 3, 'activation': 'relu', 'pool_size': 4}}],
           dtype=object)

In [83]:
mutations.mutate_network(indiv, verbose=True)

MUTATING 1 BLOCKS OF NETWORK
MUTATED LAYER dropout
REMOVE BLOCK ELEM:  3
NEW BLOCK ELEM:  2


(Individual([{'type': 'dropout', 'params': {'rate': 0.19}},
             {'type': 'conv2ddropout', 'params': {'filters': 16, 'kernel_size': 3, 'activation': 'relu', 'rate': 0.29}}],
            dtype=object),)